# Reed-Solomon Encoding

Reed-Solomon encoding is a type of error-correcting code. It takes original data and encodes it using a generator polynomial to produce redundant check symbols. We can take those check symbols and correct errors in transmitted data up to a certain limit.

Let $\mathbb{F}_q$ be a finite field with $q$ elements, where $q$ is a prime power. An RS code is defined by the parameters $(n, k)$, where $n$ and $k$ are positive integers such that $1 \leq k \leq n \leq q$.

The encoding process is as follows:

1. The original data is divided into $k$ message symbols, each representing an element of $\mathbb{F}_q$.
2. A generator polynomial $g(x)$ of degree $n-k$ is used to compute $n-k$ check symbols.
3. The final codeword consists of the $k$ message symbols and the $n-k$ check symbols, forming a vector of length $n$.

The generator polynomial $g(x)$ is defined as: $$g(x) = \prod_{i=1}^{n-k} (x - \alpha^{i})$$, where $\alpha$ is a primitive element of $\mathbb{F}_q$.

The encoding process can be represented as: $$\mathbf{c} = \left(c_0, c_1, \dots, c_{n-1}\right) = \left(m_0, m_1, \dots, m_{k-1}, c_k, c_{k+1}, \dots, c_{n-1}\right)$$, where $\mathbf{m} = (m_0, m_1, \dots, m_{k-1})$ is the message vector, and $\mathbf{c}$ is the codeword vector.

The properties of an RS code include the ability to detect and correct up to $t = \lfloor \frac{n-k}{2} \rfloor$ errors in the received codeword.

In [1]:
from galois import GF

# Create GF(256) field
field = GF(2**8)
alpha = field.primitive_element


def encode(message, n, k):
    # Convert message to field elements
    message = [field(m) for m in message]

    # Step 1: Generate the generator polynomial
    # g(x) = (x + α)(x + α²)...(x + α^(n-k))
    roots = [alpha**i for i in range(1, n - k + 1)]
    g = [field(1)]  # g(x) starts as 1

    for root in roots:
        # Multiply g(x) by (x + root)
        g_new = [field(0)] * (len(g) + 1)
        for i in range(len(g)):
            g_new[i] ^= g[i]  # x * g[i]
            g_new[i + 1] ^= g[i]  # g[i]
            g_new[i] ^= g[i] * root  # g[i] * root
        g = g_new

    print(f"Generator polynomial: {[int(x) for x in g]}")

    # Step 2: Encode message
    # First multiply message by x^(n-k)
    codeword = [field(x) for x in message] + [field(0)] * (n - k)

    # Divide by generator polynomial to get remainder
    dividend = codeword.copy()
    for i in range(k):
        if dividend[i] != 0:
            coef = dividend[i]
            for j in range(len(g)):
                dividend[i + j] ^= coef * g[j]

    # Combine message and remainder
    for i in range(k, n):
        codeword[i] = dividend[i]

    return [int(x) for x in codeword]


def decode(received, n, k):
    # Convert to field elements
    received = [field(x) for x in received]

    # Step 1: Calculate syndromes
    syndromes = []
    for i in range(1, n - k + 1):
        power = alpha**i
        syndrome = field(0)
        for j, r in enumerate(received):
            syndrome ^= r * (power**j)
        syndromes.append(int(syndrome))

    print(f"Syndromes: {syndromes}")

    # If all syndromes are zero, no errors
    if all(s == 0 for s in syndromes):
        return [int(x) for x in received[:k]]

    # For simplicity, we'll only handle single errors
    # For a single error at position j with value e:
    # S₁ = e·α^j
    # S₂ = e·(α^j)²
    # Therefore, S₂/S₁ = α^j, which gives us the error position
    if syndromes[0] != 0 and syndromes[1] != 0:
        # Find error position
        error_locator = field(syndromes[1]) / field(syndromes[0])
        error_pos = 0
        x = field(1)
        while x != error_locator and error_pos < n:
            x *= alpha
            error_pos += 1

        if error_pos < n:
            print(f"Error found at position {error_pos}")
            # Calculate error value
            error_value = field(syndromes[0]) / (alpha**error_pos)
            received[error_pos] ^= error_value

    return [int(x) for x in received[:k]]


def run():
    n, k = 7, 3
    message = [5, 2, 3]
    print(f"Original message: {message}")
    codeword = encode(message, n, k)
    print(f"Encoded codeword: {codeword}")
    received = codeword.copy()
    received[4] ^= 1
    print(f"Received (with error): {received}")
    decoded = decode(received, n, k)
    print(f"Decoded message: {decoded}")
    if decoded == message:
        print("Successfully corrected error!")
    else:
        print("Failed to correct error")


run()

Original message: [5, 2, 3]
Generator polynomial: [0, 0, 0, 0, 0]
Encoded codeword: [5, 2, 3, 0, 0, 0, 0]
Received (with error): [5, 2, 3, 0, 1, 0, 0]
Syndromes: [29, 32, 24, 78]
Decoded message: [5, 2, 3]
Successfully corrected error!
